In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from thefuzz import fuzz
from tqdm.notebook import tqdm
import time
import json

In [2]:
def create_driver(options=None):
    # Set up Selenium options
    if options is None:
        options = Options()
        # options.add_argument("--headless")  # Run in headless mode if you don't need to see the browser
        options.add_argument(
            "--disable-blink-features=AutomationControlled"
        )  # Avoid detection

    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")
    # options.add_argument("--start-maximized")

    # Path to your ChromeDriver

    # Create a WebDriver instance
    cService = webdriver.ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=cService, options=options)
    return driver

In [3]:
keyword = "เจ๊ม่วย ก๋วยเตี๋ยวเป็ดสุดซอย"
weblink = f"https://www.google.com/maps/search/{keyword}/@12.5596999, 99.9615208,17z"
weblink = "https://www.google.com/maps/search/ร้านเจ้ม๋วยก๋วยเตี๋ยวเป็ด/@14.58315600800006,100.99632440300007,13z"
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--force-device-scale-factor=1.5")

In [4]:
driver = create_driver(options)
# Open the website
driver.get(weblink)

# Wait for Cloudflare's challenge to be bypassed
time.sleep(1)  # Adjust the sleep time as needed based on the challenge duration

# Example: Scraping the page title
page_title = driver.title
# print(f"Page Title: {page_title}")

# Example: Scraping specific elements
feeds = driver.find_elements(By.XPATH, "//div[@role='feed']")
poi_title = None
if len(feeds) > 0:
    feed_divs = feeds[0].find_elements(
        By.XPATH,
        "//div[contains(@jsaction, 'mouseover:pane') and contains(@jsaction, 'mouseout:pane')]",
    )
    count = 0

    for feed_div in feed_divs:
        if count == 1:
            feed_div.click()
            break

        if keyword in feed_div.text:
            count += 1

    poi_title = feed_divs[0].text

review_starts = driver.find_elements(
    By.XPATH, "//div[contains(@jsaction, '.moreReviews')]"
)
if len(review_starts) > 0:
    review_starts[0].click()

anchor_tag = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, "//div[@role='tablist']"))
)
button_tags = anchor_tag.find_elements(By.TAG_NAME, "button")

dead_letter = 0
while len(button_tags) == 2 and dead_letter <= 10:
    time.sleep(0.2)
    button_tags = anchor_tag.find_elements(By.TAG_NAME, "button")
    dead_letter += 1

if len(button_tags) == 3:
    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center'});", button_tags[1]
    )
    button_tags[1].click()
if len(button_tags) == 4:
    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center'});", button_tags[2]
    )
    button_tags[2].click()

relevant_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, "//button[@aria-label='เกี่ยวข้องที่สุด']"))
)
relevant_element.click()

menu_items = []
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, "//div[@role='menuitemradio']"))
)
menu_items = driver.find_elements(By.XPATH, "//div[@role='menuitemradio']")
menu_items[1].click()

prev_len_scroll_element = -1
sleep = 0.2
scroll_elements = []
while True:
    jslog = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located(
            (By.XPATH, "//div[@jslog='26354;mutable:true;']")
        )
    )
    scroll_elements = jslog.find_elements(By.TAG_NAME, "span")
    if len(scroll_elements) == prev_len_scroll_element:
        div_elements = jslog.find_elements(By.TAG_NAME, "div")
        if div_elements[-2].text == " " or sleep > 120:
            break
        time.sleep(sleep)
        sleep = sleep * 2
        continue
    sleep = 0.2
    driver.execute_script("arguments[0].scrollIntoView(true);", scroll_elements[-1])
    prev_len_scroll_element = len(scroll_elements)
    time.sleep(0.2)

review_elements = driver.find_elements(By.CSS_SELECTOR, "div.fontBodyMedium")
years = review_elements[-1].find_elements(By.TAG_NAME, "span")
year_text = [year.text for year in years]

all_reviews = [i.text.replace("\n", "|").replace("\t", ";") for i in review_elements]
last_review_text = ":".join(year_text).replace("\n", "|").replace("\t", ";")
print(year_text)

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55c52f8608fa <unknown>
#1 0x55c52f371d20 <unknown>
#2 0x55c52f3a9438 <unknown>
#3 0x55c52f3a4f96 <unknown>
#4 0x55c52f3f0d86 <unknown>
#5 0x55c52f3f03d6 <unknown>
#6 0x55c52f3e4893 <unknown>
#7 0x55c52f3b330d <unknown>
#8 0x55c52f3b432e <unknown>
#9 0x55c52f82d00b <unknown>
#10 0x55c52f830f97 <unknown>
#11 0x55c52f81971c <unknown>
#12 0x55c52f831b17 <unknown>
#13 0x55c52f7fe6cf <unknown>
#14 0x55c52f84f6b8 <unknown>
#15 0x55c52f84f880 <unknown>
#16 0x55c52f85f776 <unknown>
#17 0x7fe3b2d10609 start_thread


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(
    service=Service("/usr/local/bin/chromedriver"),
)

driver = webdriver.Chrome()
driver.get("https://www.google.com")
print(driver.title)
driver.quit()

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x565425b0ba63 <unknown>
#1 0x565425880458 <unknown>
#2 0x5654258a32e3 <unknown>
#3 0x56542589ee9f <unknown>
#4 0x5654258d8b72 <unknown>
#5 0x5654258d2fd3 <unknown>
#6 0x5654258a9514 <unknown>
#7 0x5654258aa505 <unknown>
#8 0x565425b37e2e <unknown>
#9 0x565425b4d886 <unknown>
#10 0x565425b38d75 <unknown>
#11 0x565425b4ed94 <unknown>
#12 0x565425b2f8eb <unknown>
#13 0x565425b69cd8 <unknown>
#14 0x565425b69e58 <unknown>
#15 0x565425b83dfd <unknown>
#16 0x7f7410d3b609 start_thread


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

# options.headless = True

driver = webdriver.Chrome(service="/usr/bin/chromedriver", options=options)

driver.get("https://google.com/")
print(driver.title)
driver.quit()

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")  # Required in some Linux environments
chrome_options.add_argument("--disable-gpu")  # Avoid GPU errors in headless mode
chrome_options.add_argument("--disable-dev-shm-usage")  # Use /tmp instead of /dev/shm
chrome_options.add_argument("--remote-debugging-port=9222")  # Enable debugging
chrome_driver_path = "/usr/local/bin/chromedriver"

service = Service("/usr/local/bin/chromedriver")  # Update path if necessary
driver = webdriver.Chrome(
    service=service,
    options=chrome_options,
)

driver.get("https://www.google.com")
print("Page title:", driver.title)
driver.quit()

TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [256]:
len(review_elements)

265

In [7]:
driver.quit()

NameError: name 'driver' is not defined